In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Libraries Required**

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge # Ridge regression model
import tensorflow as tf
from wordcloud import STOPWORDS
from sklearn.model_selection import train_test_split,KFold
import warnings
warnings.filterwarnings("ignore")

**Datasources**

In [ ]:
prev_train_df = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
test = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
sample = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
target = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
test=pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

**Custom weights to existing columns**

In [ ]:
prev_train_df["severe_toxic"] = prev_train_df["severe_toxic"] * 4.5
prev_train_df["threat"] = prev_train_df["threat"] *5.0
prev_train_df["identity_hate"] = prev_train_df["identity_hate"] * 4.0

prev_train_df["total_toxic"] = prev_train_df[["toxic", "severe_toxic", "obscene", "threat", "insult",
                                              "identity_hate"]].sum(axis = 1).astype(np.int64)

In [ ]:
train_df = prev_train_df[["comment_text", "total_toxic"]]

**Data cleaning--1**

In [ ]:
train_df["Comment"] = train_df['comment_text'].str.replace('[^\w\s\n]','')
train_df.head()

In [ ]:
train_df.drop(['comment_text'],axis=1)

In [ ]:
text=train_df.loc[1:70001,['Comment']]
target=train_df.loc[1:70000,['total_toxic']]

In [ ]:
train_df['text'] = train_df['Comment'].apply(lambda x:x.lower())

**Data cleaning --2**

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [ ]:
#removing the stop words from the corpus
train_df['filter1'] =train_df['text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
train_df['filter1']

**Vectorization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvec = TfidfVectorizer(analyzer = 'char_wb', ngram_range = (3,5))
tfv = tfvec.fit_transform(train_df["filter1"])

**Data selection**

In [ ]:
X=tfv
Y=train_df['total_toxic']

**Splitting data into train and test**

**Ridge modelling**

In [ ]:
rr_model = Ridge(alpha=0.5)
rr_model.fit(X,Y)

In [ ]:
tfv_comments = tfvec.transform(test["text"])
pred = rr_model.predict(tfv_comments)

In [ ]:
sub = pd.DataFrame()
sub["comment_id"] = test["comment_id"]
sub["score"] = pred*200
sub.to_csv('submission.csv',index=False)
sub